## Second heuristic - Preliminary implementation

### Description

If there is a deposit and a withdraw transaction with **unique** gas prices (e.g., 3.1415926 Gwei), then we consider the deposit and the withdraw transactions linked. The corresponding deposit transaction can be removed from any other withdraw transaction’s anonymity set.

In [7]:
using DataFrames
using CSV
using Pkg; Pkg.add("ProgressBars")
using ProgressBars

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
   Installed ProgressBars ─ v1.4.0
    Updating `~/tornado_cash_anonymity_tool/Project.toml`
  [49802e3a] + ProgressBars v1.4.0
    Updating `~/tornado_cash_anonymity_tool/Manifest.toml`
  [49802e3a] + ProgressBars v1.4.0
Precompiling project...
  ✓ ProgressBars
1 dependency successfully precompiled in 3 seconds (176 already precompiled)


In [3]:
withdraw_transactions_df = CSV.read("../data/complete_withdraw_txs.csv", DataFrame)
deposit_transactions_df = CSV.read("../data/complete_deposit_txs.csv", DataFrame);

In [4]:
ENV["COLUMNS"]=10000
ENV["LINES"]=10;

### Function summary: filter_by_unique_gas_price

Filters a transaction DataFrame, leaving only the transactions (rows) that have unique gas_price within all transactions.

In [5]:
# Filters a transaction DataFrame, leaving only the rows that have unique gas_price. 
function filter_by_unique_gas_price(transactions_df)
    unique_gas_prices = filter(row -> row.count==1, combine(groupby(transactions_df, :gas_price), nrow => :count))[!, "gas_price"]
    filter(row -> row.gas_price ∈ unique_gas_prices, transactions_df)
end

filter_by_unique_gas_price (generic function with 1 method)

In [6]:
unique_gasp_deposits = filter_by_unique_gas_price(deposit_transactions_df)
first(unique_gasp_deposits, 5)

,Column1,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address
,Int64,String,Int64,Int64,String,String,Int128,Int64,Int64,String,Int64,Int64,Missing,Missing,Int64,String31,Int64,String,Float64?,Float64?,Float64?,Int64,String
1,12,0xbd83053f8afa7777f54a4aca6b8e112fa31b888922dc5b9a9a65eb66e9a6996f,7,63,0x6c6e4816ecfa4481472ff88f32a3e00f2eaa95a1,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,100000000000000000,800000,30838446643,0xb214faa527a20ba920c8ae877d67ce1ebd7420dafb3150e001eca78166fd6d66a5fd253e,6222489,800000,missing,missing,0,2020-05-27 03:30:44 UTC,10145408,0x837b3482443f027f6f045644bf002243f72304686015a2d6265676b2a2fc630b,missing,missing,missing,30838446643,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc
2,132,0x6c416af65ea3a4bc096663c94f5b1fb0cba91607f61703657094f1f5441a3a12,3,51,0x27972d10f153099b3649ea8546a11d91315455e5,0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f,0,1200000,71302125000,0xb214faa52d8f3e8b9934e70c0fbf5bb3cf355027815a11191fb5e2f1d3f6f84beb2a7c35,4108630,992258,missing,missing,1,2020-09-26 08:10:08 UTC,10937092,0x32f0f0fd04d3af8210d2eb956fdec21e09ebff5a55209c0a2c559bb1c034b158,missing,missing,missing,71302125000,0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f
3,1549,0x830dbd534d13cd43cb078b7cad8a9c5137bb19aa8bf38e0c3b0e222b688d8340,0,63,0x43eefeb3db479e7b22e015572f38b6af633a43ff,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,1000000000000000000,275000,595000000000,0xb214faa500fe7a849e7374f033604417bf6ccd369ea65e027f5327f58fcb8667cb469769,4390453,274947,missing,missing,0,2020-09-17 20:50:21 UTC,10881994,0x5270afd78906cc7264620b2c6fbf8c9221cf8053c8111edc0d7457966340dd81,missing,missing,missing,595000000000,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936
4,1582,0x56837665efa6b3ed43685c79744f8b3c575090f7d076f7187c2ae3ace5aea0c9,8,41,0x0fcdae4a4aeb0497f39a209009dc200a6466b7bd,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,10000000000000000000,1200000,5231046931,0xb214faa52456540e731a7caef51cd40086cc0236eb70b69160587d15e753084dbb4df3c5,5058769,978691,missing,missing,1,2020-01-24 05:54:15 UTC,9342713,0x2263281e65fb7d50125309a90864131650ca560ccd51c3e8e2847fbd2a18ee9f,missing,missing,missing,5231046931,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf
5,1690,0x038303d14327a2437bd789a70c0f7daa95b3ab89b46af0418fd0f677bdf2766c,82,163,0x3eb24c61590e6cd6fc26cf4fb938316206147859,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,100000000000000000,1200000,57151624549,0xb214faa517409b6dc9fc6d2da0fd00fd2ea324a61922aee92d412c3e1bf394b19d91ffa9,9371665,978691,missing,missing,1,2020-10-16 09:18:01 UTC,11066064,0xfa752cf56aaec471375e2996736d20444b146199e2c3b84851cb16c65695cd3f,missing,missing,missing,57151624549,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc


### Function summary: same_gas_price_heuristic

This function receives a particular withdraw transaction and a DataFrame with the unique gas price deposits.

It returns a tuple:
* $(true, deposit$ $hash)$ when a deposit transaction with the same gas price as the withdrawal transaction is found.
* $(false, nothing)$ when such a deposit is not found.

In [8]:
# Given a withdrawal transaction and a DataFrame with unique gas_price deposit transactions, checks
# if there is a deposit transaction with the same gas_price as the withdrawal transaction.
function same_gas_price_heuristic(withdrawal_transaction, unique_gas_price_deposit_dataframe)
    for row in eachrow(unique_gas_price_deposit_dataframe)
        if withdrawal_transaction.gas_price == row.gas_price && withdrawal_transaction.block_timestamp > row.block_timestamp
            return (true, row.hash)
        end
    end
    (false, nothing)
end

same_gas_price_heuristic (generic function with 1 method)

In [9]:
same_gas_price_heuristic(withdraw_transactions_df[1,:], unique_gasp_deposits)

(false, nothing)

### Function summary: apply_same_gas_price_heuristic

Applies the heuristic to all the withdraw_transactions DataFrame. Returns a dicionary mapping linked withdrawal and deposit transaction hashes.

In [10]:
# Applies the function to detect same unique gas_prices to all the withdraw_transactions data.
# Returns a list of tuples, each tuple with the deposit transaction hash in the first index and
# the withdrawal transaction hash in the second index.
function apply_same_gas_price_heuristic(deposit_dataframe, withdraw_dataframe)
    unique_gas_price_deposits = filter_by_unique_gas_price(deposit_dataframe)
    withdrawal_to_deposit = []
    for withdraw_row in ProgressBar(eachrow(withdraw_dataframe))
        same_gas_deposit_hash = same_gas_price_heuristic(withdraw_row, unique_gas_price_deposits)
        if same_gas_deposit_hash[1]
            push!(withdrawal_to_deposit, (same_gas_deposit_hash[2], withdraw_row.hash))
        end
    end
    withdrawal_to_deposit
end 

apply_same_gas_price_heuristic (generic function with 1 method)

In [11]:
@time link_same_gas = apply_same_gas_price_heuristic(deposit_transactions_df, withdraw_transactions_df);

0.0%┣                                        ┫ 0/83.8k [00:00<-1:-9:-49, -0s/it]
0.0%┣                                        ┫ 1/83.8k [00:00<Inf:Inf, InfGs/it]
0.0%┣                                         ┫ 8/83.8k [00:00<01:10:21, 20it/s]
0.0%┣                                           ┫ 13/83.8k [00:00<52:23, 27it/s]
0.0%┣                                           ┫ 39/83.8k [00:01<18:26, 76it/s]
0.1%┣                                          ┫ 61/83.8k [00:01<12:52, 108it/s]
0.1%┣                                          ┫ 87/83.8k [00:01<09:49, 142it/s]
0.1%┣                                         ┫ 112/83.8k [00:01<08:15, 169it/s]
0.2%┣                                         ┫ 134/83.8k [00:01<07:26, 188it/s]
0.2%┣                                         ┫ 153/83.8k [00:01<06:58, 200it/s]
0.2%┣                                         ┫ 179/83.8k [00:01<06:21, 219it/s]
0.2%┣                                         ┫ 203/83.8k [00:01<05:57, 234it/s]
0.3%┣                       

3.0%┣█▏                                      ┫ 2.5k/83.8k [00:06<03:08, 433it/s]
3.0%┣█▏                                      ┫ 2.5k/83.8k [00:06<03:07, 434it/s]
3.1%┣█▎                                      ┫ 2.6k/83.8k [00:06<03:06, 436it/s]
3.1%┣█▎                                      ┫ 2.6k/83.8k [00:06<03:06, 436it/s]
3.1%┣█▎                                      ┫ 2.6k/83.8k [00:06<03:05, 438it/s]
3.2%┣█▎                                      ┫ 2.6k/83.8k [00:06<03:06, 436it/s]
3.2%┣█▎                                      ┫ 2.7k/83.8k [00:06<03:06, 437it/s]
3.2%┣█▎                                      ┫ 2.7k/83.8k [00:06<03:05, 438it/s]
3.3%┣█▎                                      ┫ 2.7k/83.8k [00:06<03:05, 438it/s]
3.3%┣█▎                                      ┫ 2.8k/83.8k [00:06<03:05, 439it/s]
3.3%┣█▎                                      ┫ 2.8k/83.8k [00:06<03:04, 440it/s]
3.4%┣█▍                                      ┫ 2.8k/83.8k [00:06<03:04, 441it/s]
3.4%┣█▍                     

5.7%┣██▎                                     ┫ 4.8k/83.8k [00:11<03:06, 424it/s]
5.7%┣██▎                                     ┫ 4.8k/83.8k [00:11<03:06, 424it/s]
5.8%┣██▎                                     ┫ 4.8k/83.8k [00:11<03:06, 425it/s]
5.8%┣██▎                                     ┫ 4.8k/83.8k [00:11<03:06, 425it/s]
5.8%┣██▎                                     ┫ 4.9k/83.8k [00:11<03:06, 423it/s]
5.8%┣██▍                                     ┫ 4.9k/83.8k [00:12<03:06, 424it/s]
5.9%┣██▍                                     ┫ 4.9k/83.8k [00:12<03:06, 424it/s]
5.9%┣██▍                                     ┫ 4.9k/83.8k [00:12<03:06, 424it/s]
5.9%┣██▍                                     ┫ 5.0k/83.8k [00:12<03:06, 424it/s]
5.9%┣██▍                                     ┫ 5.0k/83.8k [00:12<03:06, 424it/s]
6.0%┣██▍                                     ┫ 5.0k/83.8k [00:12<03:06, 425it/s]
6.0%┣██▍                                     ┫ 5.0k/83.8k [00:12<03:05, 425it/s]
6.0%┣██▍                    

8.5%┣███▍                                    ┫ 7.1k/83.8k [00:17<02:59, 427it/s]
8.6%┣███▍                                    ┫ 7.2k/83.8k [00:17<02:59, 428it/s]
8.6%┣███▍                                    ┫ 7.2k/83.8k [00:17<02:59, 428it/s]
8.6%┣███▌                                    ┫ 7.2k/83.8k [00:17<02:59, 428it/s]
8.7%┣███▌                                    ┫ 7.3k/83.8k [00:17<02:59, 428it/s]
8.7%┣███▌                                    ┫ 7.3k/83.8k [00:17<02:59, 428it/s]
8.7%┣███▌                                    ┫ 7.3k/83.8k [00:17<02:58, 429it/s]
8.8%┣███▌                                    ┫ 7.3k/83.8k [00:17<02:58, 429it/s]
8.8%┣███▌                                    ┫ 7.4k/83.8k [00:17<02:58, 430it/s]
8.8%┣███▌                                    ┫ 7.4k/83.8k [00:17<02:58, 430it/s]
8.9%┣███▌                                    ┫ 7.4k/83.8k [00:17<02:58, 430it/s]
8.9%┣███▋                                    ┫ 7.5k/83.8k [00:17<02:57, 430it/s]
8.9%┣███▋                   

11.4%┣████▍                                  ┫ 9.5k/83.8k [00:22<02:53, 430it/s]
11.4%┣████▌                                  ┫ 9.6k/83.8k [00:22<02:53, 430it/s]
11.4%┣████▌                                  ┫ 9.6k/83.8k [00:22<02:52, 430it/s]
11.5%┣████▌                                  ┫ 9.6k/83.8k [00:22<02:52, 430it/s]
11.5%┣████▌                                  ┫ 9.6k/83.8k [00:22<02:52, 430it/s]
11.5%┣████▌                                  ┫ 9.6k/83.8k [00:22<02:53, 430it/s]
11.5%┣████▌                                  ┫ 9.7k/83.8k [00:22<02:52, 430it/s]
11.6%┣████▌                                  ┫ 9.7k/83.8k [00:23<02:52, 430it/s]
11.6%┣████▌                                  ┫ 9.7k/83.8k [00:23<02:52, 430it/s]
11.6%┣████▌                                  ┫ 9.7k/83.8k [00:23<02:52, 430it/s]
11.7%┣████▌                                  ┫ 9.8k/83.8k [00:23<02:52, 430it/s]
11.7%┣████▌                                  ┫ 9.8k/83.8k [00:23<02:52, 430it/s]
11.7%┣████▋                 

...

91.8%┣██████████████████████████████████▉   ┫ 76.9k/83.8k [02:46<00:15, 463it/s]
91.8%┣███████████████████████████████████   ┫ 76.9k/83.8k [02:46<00:15, 462it/s]
91.9%┣███████████████████████████████████   ┫ 77.0k/83.8k [02:46<00:15, 462it/s]
91.9%┣███████████████████████████████████   ┫ 77.0k/83.8k [02:46<00:15, 462it/s]
91.9%┣███████████████████████████████████   ┫ 77.0k/83.8k [02:47<00:15, 463it/s]
92.0%┣███████████████████████████████████   ┫ 77.0k/83.8k [02:47<00:15, 462it/s]
92.0%┣███████████████████████████████████   ┫ 77.1k/83.8k [02:47<00:15, 462it/s]
92.0%┣███████████████████████████████████   ┫ 77.1k/83.8k [02:47<00:14, 463it/s]
92.0%┣███████████████████████████████████   ┫ 77.1k/83.8k [02:47<00:14, 463it/s]
92.1%┣███████████████████████████████████   ┫ 77.1k/83.8k [02:47<00:14, 463it/s]
92.1%┣███████████████████████████████████   ┫ 77.2k/83.8k [02:47<00:14, 463it/s]
92.1%┣███████████████████████████████████   ┫ 77.2k/83.8k [02:47<00:14, 462it/s]
92.2%┣██████████████████████

94.7%┣████████████████████████████████████  ┫ 79.3k/83.8k [02:52<00:10, 462it/s]
94.7%┣████████████████████████████████████  ┫ 79.4k/83.8k [02:52<00:10, 462it/s]
94.8%┣████████████████████████████████████  ┫ 79.4k/83.8k [02:52<00:09, 462it/s]
94.8%┣████████████████████████████████████  ┫ 79.4k/83.8k [02:52<00:09, 462it/s]
94.8%┣████████████████████████████████████  ┫ 79.4k/83.8k [02:52<00:09, 462it/s]
94.8%┣████████████████████████████████████  ┫ 79.5k/83.8k [02:52<00:09, 462it/s]
94.9%┣████████████████████████████████████  ┫ 79.5k/83.8k [02:52<00:09, 462it/s]
94.9%┣████████████████████████████████████  ┫ 79.5k/83.8k [02:52<00:09, 462it/s]
94.9%┣████████████████████████████████████  ┫ 79.5k/83.8k [02:52<00:09, 462it/s]
95.0%┣████████████████████████████████████  ┫ 79.6k/83.8k [02:52<00:09, 462it/s]
95.0%┣████████████████████████████████████  ┫ 79.6k/83.8k [02:52<00:09, 462it/s]
95.0%┣████████████████████████████████████▏ ┫ 79.6k/83.8k [02:52<00:09, 462it/s]
95.1%┣██████████████████████

97.5%┣█████████████████████████████████████ ┫ 81.7k/83.8k [02:57<00:04, 461it/s]
97.6%┣█████████████████████████████████████ ┫ 81.7k/83.8k [02:57<00:04, 461it/s]
97.6%┣█████████████████████████████████████ ┫ 81.8k/83.8k [02:57<00:04, 461it/s]
97.6%┣█████████████████████████████████████ ┫ 81.8k/83.8k [02:57<00:04, 461it/s]
97.6%┣█████████████████████████████████████ ┫ 81.8k/83.8k [02:57<00:04, 461it/s]
97.7%┣█████████████████████████████████████▏┫ 81.8k/83.8k [02:57<00:04, 461it/s]
97.7%┣█████████████████████████████████████▏┫ 81.9k/83.8k [02:58<00:04, 461it/s]
97.7%┣█████████████████████████████████████▏┫ 81.9k/83.8k [02:58<00:04, 461it/s]
97.7%┣█████████████████████████████████████▏┫ 81.9k/83.8k [02:58<00:04, 461it/s]
97.8%┣█████████████████████████████████████▏┫ 81.9k/83.8k [02:58<00:04, 461it/s]
97.8%┣█████████████████████████████████████▏┫ 81.9k/83.8k [02:58<00:04, 461it/s]
97.8%┣█████████████████████████████████████▏┫ 82.0k/83.8k [02:58<00:04, 461it/s]
97.9%┣██████████████████████

184.027105 seconds (4.88 G allocations: 72.912 GiB, 9.07% gc time)


100.0%┣█████████████████████████████████████┫ 83.8k/83.8k [03:01<00:00, 462it/s]
100.0%┣█████████████████████████████████████┫ 83.8k/83.8k [03:01<00:00, 462it/s]
100.0%┣█████████████████████████████████████┫ 83.8k/83.8k [03:01<00:00, 462it/s]


In [12]:
# A DataFrame with the linked transactions.
linked_transactions_df = DataFrame("deposit_hash"=>[tuple[1] for tuple in link_same_gas],"withdrawl_hash"=>[tuple[2] for tuple in link_same_gas])

,deposit_hash,withdrawl_hash
,String,String
1,0x84508c35b90983284f9605117333ed9c379be64bf82f3c04fd7338a1a3b5dc62,0x6b1ffed49640dce414b1110fb8111c6e6d5cae0707db526b4628d1088a0ae0be
2,0x5018fe2c5bcc2a062357edf5f367a9237c7b332a937646ba3489bf3a9b6fcc9a,0x0f742f92162f7a30c07d314a891acebae3baf229a667ae571dd501d25fa61528
3,0x3156e68753b1d12cd68da86a45f1f2b96b48c73c3d982b193c75b4cee0e7a44c,0xf6b761a8d27dddf0710b4cc2c0c5f8a1d740b5500cd62021c7f91dba05aba966
4,0x8c72a5f48918f1c6a79ce219e042ddb81f1a2442138a99c7272d4856bf58246c,0xc5205147de3a60cb321c005e59a8ea0004bcccb8389b9a48540b628ab4767f77
5,0x6a082db2993c2beaa1134cd3f266d9b79b0cfad43dbc635b4488ab64106ab452,0xc40d7044612eae0ab3986df553b0043374516332f53581480c2f9570c2faee2e
6,0x3f3f64f2c22f87dfc87ddc29db79a85bc6aac805bfce418fc36dee562cbd246f,0x59fd9fca7968b22978ea7c1a7ce2f9d586740fd7b92224145120a18d8a754a7a
7,0x3b81998758b17d407c129307c8f2ab0fd5066de0d6c9432be81602257662f4cf,0xc1d0d0aa0c0d1581ee8e92ed8167e9fc08c12769a68fa50b71d504cd115ebae3
8,0x5018fe2c5bcc2a062357edf5f367a9237c7b332a937646ba3489bf3a9b6fcc9a,0x60bd987f24c03d889efeb6c0adc7ccea325d9237b8890f66623732515743c997
9,0x5018fe2c5bcc2a062357edf5f367a9237c7b332a937646ba3489bf3a9b6fcc9a,0xc14dbb973326e2b20f8d5ba9cf3767e05249afd3be551db3a34545a0dd27b47c
